In [ ]:
%reload_ext autoreload
%autoreload 2

# 模拟器爬虫
> 解决一些复杂问题，或者通用爬虫，虽然性能不佳，但看看能做到啥程度

本质上这是个自动化测试的工具，实现如按键精灵一样的效果，但可能也有瓶颈，也会被识别为爬虫


* [官方介绍](https://www.selenium.dev/documentation/en/introduction/)
* [API文档](https://www.selenium.dev/selenium/docs/api/py/api.html)

感觉研究这个意义不大，因为还是有被屏蔽的问题，越是有价值的内容，反爬措施越严格，甚至到需要录制真人操作的地步，那就有点背离爬虫的初衷，只是做长尾的内容api化，不需要爬哪些最难的，反而是找到一个更需要网页api的场景，提供更智能的响应


In [ ]:
# export
import pandas as pd
import requests,json,re
from bs4 import BeautifulSoup,Tag,NavigableString
from collections import Counter
from crawler_from_scratch.utils import *

## Selenium

[selenium-python文档](https://selenium-python.readthedocs.io/)


下载chromedriver https://sites.google.com/a/chromium.org/chromedriver/downloads

In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
driver = Chrome(executable_path='./chromedriver')
driver.get('http://www.baidu.com')

### 如何找到一个元素并显示它的值
XPath语法 https://www.runoob.com/xpath/xpath-syntax.html

In [ ]:
nav = driver.find_element_by_xpath("//div[@id='u1']/a[2]")
nav.text

'新闻'

### 跳转一个页面并获取当前页url

In [ ]:
kw = driver.find_element_by_id('kw')
kw.send_keys('hawk',Keys.ENTER)

element = WebDriverWait(driver, 10).until(
    lambda driver: driver.find_element_by_id('content_left')
)
driver.current_url

'https://www.baidu.com/s?ie=utf-8&f=8&rsv_bp=1&rsv_idx=1&tn=baidu&wd=hawk&rsv_t=f672jDJ7kro9rT2C6vRNNbm7Y8Hsz02eZktZCxrKTlbtZItRdPTr%2FGMybp0&rsv_enter=1&rsv_dl=tb&rsv_sug3=4&rsv_sug2=0&inputT=97&rsv_sug4=97'

### 滚动到页面底部,加载更多内容

In [ ]:
driver.get('https://www.ixigua.com/')
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

### 如何查看过程中获取的network files
> 需要搭配proxy

## Appium

* [官网](http://appium.io/docs/en/about-appium/getting-started/)
* [Appium Desktop](https://github.com/appium/appium-desktop/releases)
* [Python Client](https://github.com/appium/python-client) 选dmg

无法打开的问题 https://github.com/appium/appium-desktop/issues/839



In [ ]:
# !pip install Appium-Python-Client

from appium import webdriver as appdriver

### Real Device Setup
For Android automation, no additional setup is required for testing on real devices, other than these simple requirements:

* Ensure that Developer mode is turned on for the device.
* Ensure that the device is connected via USB to the Appium host, and can be seen by ADB (run adb devices to make sure).
* Ensure that "Verify Apps" in settings is disabled, to allow Appium's helper apps to function without manual intervention.
* (For some specific commands, the device might need to be rooted, though this is not the norm.)

In [ ]:
# !brew cask install android-platform-tools
# !adb kill-server && adb server && adb shell
!adb devices 

List of devices attached
emulator-5554	device



In [ ]:
!adb shell pm list packages

In [ ]:
desired_caps = {
                    'platformName': 'Android',
                    'deviceName': '127.0.0.1:7555',
                    'platformVersion': '6.0.1',
                    # apk包名
                    'appPackage': 'com.tencent.mobileqq',
                    # apk的launcherActivity
                    'appActivity': 'com.tencent.mobileqq.activity.SplashActivity'
                    }
driver = appdriver.Remote('http://127.0.0.1:4723/wd/hub', desired_caps)

WebDriverException: Message: An unknown server-side error occurred while processing the command. Original error: JAVA_HOME is not set currently. Please set JAVA_HOME.


In [ ]:
res = requests.get('https://ncov.dxy.cn/ncovh5/view/pneumonia')

In [ ]:
re.search(r'getAreaStat = (.*)}',res.content.decode()).group()

'getAreaStat = [{"provinceName":"湖北省","provinceShortName":"湖北","currentConfirmedCount":1283,"confirmedCount":67802,"suspectedCount":0,"curedCount":63326,"deadCount":3193,"comment":"","locationId":420000,"statisticsData":"https://file1.dxycdn.com/2020/0223/618/3398299751673487511-135.json","cities":[{"cityName":"武汉","currentConfirmedCount":1279,"confirmedCount":50007,"suspectedCount":0,"curedCount":46175,"deadCount":2553,"locationId":420100},{"cityName":"孝感","currentConfirmedCount":1,"confirmedCount":3518,"suspectedCount":0,"curedCount":3389,"deadCount":128,"locationId":420900},{"cityName":"襄阳","currentConfirmedCount":1,"confirmedCount":1175,"suspectedCount":0,"curedCount":1135,"deadCount":39,"locationId":420600},{"cityName":"宜昌","currentConfirmedCount":1,"confirmedCount":931,"suspectedCount":0,"curedCount":894,"deadCount":36,"locationId":420500},{"cityName":"仙桃","currentConfirmedCount":1,"confirmedCount":575,"suspectedCount":0,"curedCount":552,"deadCount":22,"locationId":429004},{"city

## Proxy
> mitmproxy
baseproxy
模拟器里执行一个动作，proxy抓下来进一步处理